## Channels
|**N**          |0  |1  |2  |3  |4  | 5 | 6 | 7 | 8 | 9 |
|---------------|---|---|---|---|---|---|---|---|---|---|
|**Sentinel-2A**|B02|B03|B04|B05|B06|B07|B08|B8A|B11|B12|
|**Standard**   |B  |G  |R  |RE1|RE2|RE3|N  |N2 |S1 |S2 |

## Classes:
|**N**        |1  |2  |3  |4  | 5 |
|-|-|-|-|-|-|
|**seg. mask**|open water|settlements|bair soil|forest|grassland|

## Indexes:
|short|long|type|formula|
|-|-|-|-|
LSWI|Land Surface Water Index|water|(N - S1)/(N + S1)

In [ ]:
INDEXES = {
    'BI': '((S1 + R) - (N + B))/((S1 + R) + (N + B))',
    'BNDVI': '(N - B)/(N + B)',
    'MGRVI': '(G ** 2.0 - R ** 2.0) / (G ** 2.0 + R ** 2.0)',
    'NDCI': '(RE1 - R)/(RE1 + R)',
    'NLI': '((N ** 2) - R)/((N ** 2) + R)',
}

In [ ]:
from LandcoverDataset import LandcoverDataset as LD
from utils import *
from DataVis import *
from Model import CNN, CNN_load
from UNet import UNet

In [ ]:
BATCH = 2

dl_train = LD(
    img_path="datasets/USA/train/images/", 
    mask_path="datasets/USA/train/masks/", 
    # n_random = 50,
    batch_size = BATCH,
    indexes = list(INDEXES.values())
)
dl_valid = LD(
    img_path="datasets/USA/valid/images/", 
    mask_path="datasets/USA/valid/masks/", 
    # n_random = 10,
    batch_size = BATCH,
    transforms = False,
    indexes = list(INDEXES.values())
)

In [ ]:
img, mask = dl_train.rand_samp_names()
print(tif_info(img))
print(tif_info(mask))

In [ ]:
print(f"Images: {dl_train[0][0].shape}")
print(f"Masks:  {dl_train[0][1].shape}")

In [ ]:
dl_train.plot_sample(5)

In [ ]:
dl_train.getinfo()

In [ ]:
import torch.nn as nn

INPUT_CHANNELS = 10 + 5
N_CLASSES = 5

layers = [
            nn.Conv2d(INPUT_CHANNELS, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, N_CLASSES, kernel_size=1),
            nn.Upsample(size=(512, 512), mode='nearest')
]

model = CNN(layers)

In [ ]:
model = UNet(n_channels=INPUT_CHANNELS, n_classes=N_CLASSES)

In [ ]:
num_epochs = 2

In [ ]:
learning_rate = 0.002

In [ ]:
model.train(dl_train, dl_valid, num_epochs, learning_rate)

In [ ]:
from random import choice as c
dl_valid.plot_prediction(model, c(range(10)))

In [ ]:
m_name = f"ne={epochs_sum}, hl={hidden_layers}"
model.pickle(f"models/{m_name}.torch")

In [ ]:
model.unpickle("models/model.torch")

In [ ]:
model = CNN_load("models/model.torch")